In [2]:
import sys
sys.path.append("../utils/") 
from utilities import performance_metrics
import time

from src.core import Term, Atom
from src.ilp import Language_Frame, Program_Template, Rule_Template
from src.dilp import DILP
import tensorflow as tf
import numpy as np
from src.utils import process_file, process_dir, create_table, output_rules, test_rule, train, test
from src.generate_template import create_templates
import duckdb
tf.random.set_seed(1000)
np.random.seed(1000)


2024-05-31 18:29:25.832012: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-31 18:29:26.904321: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
con = duckdb.connect(':memory:')
# enable automatic query parallelization
con.execute("PRAGMA threads=2")
# enable caching of parquet metadata
con.execute("PRAGMA enable_object_cache")

In [16]:
term_x_0 = Term(True, 'X_0')
term_x_1 = Term(True, 'X_1')
input_dir = "../examples/transitivity-fraud-rule/"
input_table = create_table(con, input_dir)
print(f'ratio of positives %f' % (100*input_table['Fraud'].sum()/len(input_table)))
target, p_e, constants, B, P, N = process_dir(input_dir)

previous table dropped
ratio of positives 50.000000
65
end of facts processing
5
end of positive examples processing
21
end negative examples processing
data is in order
[To->destination(X_0,X_1), From->orig(X_0,X_1), Fraud(X_0)]


## Templates to describe Rules to describe:

Fraud_transaction(u1, u2):- Fraud(X), pred_trans(U1, U2) #v=1 (X)

pred_trans(u1, u2) :-  Orig(Y, U1), Dest(Y, U2) #v=1 (Y)

Fraud_Chain(u1, u3) :-Fraud_transaction(u1, u2), Fraud_transaction(u2, u3)



In [17]:
print(p_e)

[To->destination(X_0,X_1), From->orig(X_0,X_1), Fraud(X_0)]


In [18]:
print(target)

Fraud_chain->orig->destination(X_0,X_1)


In [25]:
p_a, rules = create_templates(p_e[:2], target, term_x_0)
rules[target][0], Rule_Template(v=1, allow_intensional=True)#Fraud(Y) :- Fraud_chain(u1, u2), Orig(Y, u2)

Fraud_trans = Atom([term_x_0, term_x_1], 'Fraud_trans')
Pred_Transaction = Atom([term_x_0, term_x_1], 'Pred_Transaction')

p_a.append(Fraud_trans)
p_a.append(Pred_Transaction)

rules[Fraud_trans] = (Rule_Template(v=1, allow_intensional=True), None) #Fraud_trans(X, U2):-  Dest(X, U2), Fraud(X).
rules[Pred_Transaction] = (Rule_Template(v=1, allow_intensional=False), None) #Orig(Y, U1), Dest(Y, U2) #v=1 (Y)

language_frame = Language_Frame(target, p_e, constants)
program_template = Program_Template(p_a, rules, T=6)

In [26]:
print(rules)

{Fraud_chain->orig->destination(X_0,X_1): (<src.ilp.template.rule_template.Rule_Template at 0x7f7ad394c110>,
  None),
 Fraud_trans(X_0,X_1): (<src.ilp.template.rule_template.Rule_Template at 0x7f7ad394c850>,
  None),
 Pred_Transaction(X_0,X_1): (<src.ilp.template.rule_template.Rule_Template at 0x7f7a80a03c10>,
  None)}

In [ ]:
print("DILP initialisation")
dilp = DILP(language_frame, B, P, N, program_template, allow_target_recursion=True)
start_time = time.time()
dilp.train()
finish_time = time.time()
print("execution time %d" % (finish_time - start_time))

DILP initialisation
